In [ ]:
from qiskit import ClassicalRegister, QuantumCircuit, QuantumRegister

logical_qubit = QuantumRegister(3)
error_ancillas = QuantumRegister(2)
syndrome_measurements = ClassicalRegister(2, "syn")
final_measurement = ClassicalRegister(3, "meas")
qc = QuantumCircuit(logical_qubit, error_ancillas, syndrome_measurements, final_measurement)
# Encoding
qc.cx(logical_qubit[0], logical_qubit[1])
qc.cx(logical_qubit[0], logical_qubit[2])
qc.barrier()
# Test error
qc.x(logical_qubit[2])
# Syndrome detection
qc.cx(logical_qubit[0], error_ancillas[0])
qc.cx(logical_qubit[1], error_ancillas[0])
qc.cx(logical_qubit[1], error_ancillas[1])
qc.cx(logical_qubit[2], error_ancillas[1])
qc.barrier()
qc.measure(error_ancillas, syndrome_measurements)
# Syndrome correction
with qc.if_test((syndrome_measurements, 0b01)):
    qc.x(logical_qubit[0])
with qc.if_test((syndrome_measurements, 0b11)):
    qc.x(logical_qubit[1])
with qc.if_test((syndrome_measurements, 0b10)):
    qc.x(logical_qubit[2])
qc.barrier()
# Final measurement
qc.measure(logical_qubit, final_measurement)
_ = qc.draw("mpl")

In [ ]:
from qiskit import transpile
from qiskit_aer import AerSimulator

# Create simulator and run the circuit
simulator = AerSimulator()
compiled_circuit = transpile(qc, simulator)
result = simulator.run(compiled_circuit, shots=1024).result()

# Get the counts
counts = result.get_counts()
print("Measurement results:")
print(counts)